# Pole stiction

This notebook is intended to characterize the slow speed stiction problem \
at the pole by spiraling into the pole where the tracking speeds will be slower and slower. \
Craig Lage - 17-Oct-22

In [ ]:
import asyncio
import time
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

In [ ]:
logger = logging.getLogger(f"Daytime Checkout {Time.now()} UT")
logger.level = logging.DEBUG

In [ ]:
logger.info(os.environ["OSPL_URI"])
logger.info(os.environ["LSST_DDS_PARTITION_PREFIX"])

### Instantiate the control classes

In [ ]:
domain = salobj.Domain()
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)
await atcs.assert_liveliness()

## Enable LATISS and ATCS

In [ ]:
# Didn't come up.  Brought up with LOVE
await latiss.enable()
await atcs.enable()
await atcs.assert_all_enabled()

If ATCamera and/or ATOODS don't come up, see section `Quick Troubleshooting` for support commands. 

### Bias verification

In [ ]:
for i in range(5):
    await latiss.take_bias(nbias=1)
    await asyncio.sleep(2.0)

In [ ]:
await atcs.disable_dome_following()

### Now run the actual test, taking a 30 second image at each location.

In [ ]:
# Move to starting location
coord=atcs.radec_from_azel(az=270, el=45)
await atcs.slew_icrs(coord.ra, coord.dec, rot=0.0, rot_type=RotType.Physical)

In [ ]:
# Now run the actual test 
# First image is 2023042000022

az = 270.0
await latiss.take_darks(30.0, 1)
for i in range(90):
    az -= 2.0
    coord=atcs.radec_from_azel(az=az, el=45)
    await atcs.slew_icrs(coord.ra, coord.dec, rot=0.0, rot_type=RotType.Physical)
    await latiss.take_darks(30.0, 1)
for i in range(90):
    az += 2.0
    coord=atcs.radec_from_azel(az=az, el=45)
    await atcs.slew_icrs(coord.ra, coord.dec, rot=0.0, rot_type=RotType.Physical)
    await latiss.take_darks(30.0, 1)


In [ ]:
# Stop tracking
await atcs.stop_tracking()

## Shutdown all ATCS components

In [ ]:
await atcs.shutdown()

## Put LATISS in standby

In [ ]:
await latiss.standby()